In [18]:
import re
import string
import pythainlp
from pythainlp import word_tokenize
from pythainlp.corpus import wordnet
from nltk.stem.porter import PorterStemmer
from nltk.corpus import words
import nltk
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import xlrd  
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

from sklearn import preprocessing
from sklearn.model_selection import ShuffleSplit

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc

import sklearn.ensemble as ensemble
import numpy
from imblearn.ensemble import BalancedBaggingClassifier

In [19]:
p_stemmer = PorterStemmer()
# text preprocession
def split_word(text):
    
    tokens = word_tokenize(text,engine='newmm')

    # หารากศัพท์ภาษาไทย และภาษาอังกฤษ
    # English
    tokens = [p_stemmer.stem(i) for i in tokens]
    
    # Thai
    tokens_temp=[]
    for i in tokens:
        w_syn = wordnet.synsets(i)
        if (len(w_syn)>0) and (len(w_syn[0].lemma_names('tha'))>0):
            tokens_temp.append(w_syn[0].lemma_names('tha')[0])
        else:
            tokens_temp.append(i)
    
    tokens = tokens_temp
    
    # ลบตัวเลข
    tokens = [i for i in tokens if not i.isnumeric()]
    
    # ลบช่องว่าง
    tokens = [i for i in tokens if not ' ' in i]

    return tokens

def createGeneralEvaluation(file):
    f = open(file, mode='w')
    f.write("time,amount_of_data,data_type,threshold,type_class,accuracy,precision,recall,fscore,support,auc_roc,auc_pr")
    f.write("\n")
    f.close()
    

def report_performance(file,time,modelType,predictions_class_train_c, predictions_class_test_c, type_class,threshold, predict_train_prob,predict_test_prob, train_label,train_feat):
    #Evaluation class (test data)
    print("Test data of "+modelType)
    data_type_test = "Test data of "+modelType
    threshold = str(threshold)
    type_class = str(type_class)
    accuracy_test = str(accuracy_score(predictions_class_test_c, Test_Y[type_class]))
    print(type_class+" Accuracy score:", 
          accuracy_score(predictions_class_test_c, Test_Y[type_class])*100)
    print("Confusion Matrix: \n", confusion_matrix(Test_Y[type_class],predictions_class_test_c))
    precision,recall,fscore,support=score(Test_Y[type_class],predictions_class_test_c)
    precision_test = (format(precision[1]))
    recall_test = (format(recall[1]))
    fscore_test = (format(fscore[1]))
    support_test = (format(support[1]))
    auc_roc_test = str(roc_auc_score(Test_Y[type_class], predict_test_prob))
    print("PRF: \n", classification_report(Test_Y[type_class],predictions_class_test_c))
    print("Area under the curve ROC: ", roc_auc_score(Test_Y[type_class], predict_test_prob))
    lr_precision_test, lr_recall_test, _ = precision_recall_curve(Test_Y[type_class], predict_test_prob)
    auc_pr_test = str(auc(lr_recall_test, lr_precision_test))
    print("Area under the curve PR: ", auc(lr_recall_test, lr_precision_test))
    print("-----------------------------------------------------------\n")

    #Evaluation class (train data)
    print("Train data of "+modelType)
    data_type_train = "Train data of "+modelType
    accuracy_train = str(accuracy_score(predictions_class_train_c, train_label))
    print(type_class+" Accuracy score:", 
          accuracy_score(predictions_class_train_c, train_label)*100)
    print("Confusion Matrix: \n",confusion_matrix(train_label,predictions_class_train_c))
    precision,recall,fscore,support=score(train_label,predictions_class_train_c)
    precision_train = (format(precision[1]))
    recall_train  = (format(recall[1]))
    fscore_train  = (format(fscore[1]))
    support_train  = (format(support[1]))
    auc_roc_train  = str(roc_auc_score(train_label, predict_train_prob))
    print("PRF: \n",classification_report(train_label,predictions_class_train_c))
    print("Area under the curve ROC: ", roc_auc_score(train_label, predict_train_prob))
    lr_precision_train, lr_recall_train, _ = precision_recall_curve(train_label, predict_train_prob)
    auc_pr_train  = str(auc(lr_recall_train, lr_precision_train))
    print("Area under the curve PR: ", auc(lr_recall_train, lr_precision_train))
    print("___________________________________________________________________________________________________________\n")
    
    f = open(file, mode='a')
    f.write(str(time)+","+str(train_feat)+","+data_type_test+","+threshold+","+type_class+","+accuracy_test+","+precision_test+","+recall_test+","+fscore_test+","+support_test+","+auc_roc_test+","+auc_pr_test)
    f.write("\n")
    f.write(str(time)+","+str(train_feat)+","+data_type_train+","+threshold+","+type_class+","+accuracy_train+","+precision_train+","+recall_train+","+fscore_train+","+support_train+","+auc_roc_train+","+auc_pr_train)
    f.write("\n")
    
    f.close()

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import tree
def train_and_label(train_X, train_y, test_X, test_y, file, type_class, time, dataType,classifier):
    if(classifier =='randomforest'):
        model = RandomForestClassifier(n_estimators=20, n_jobs=20)
    elif(classifier =='logisticRegression'):
        model = LogisticRegression()
    elif(classifier == 'knn'):
        model = KNeighborsClassifier(n_neighbors=3)
    elif(classifier == 'tree'):
        model = tree.DecisionTreeClassifier()
    elif(classifier == 'mlp'):
        model = MLPClassifier()
    elif(classifier == 'svm'):
        model = LinearSVC()
        
    model = model.fit(train_X, train_y)
    
    # eval
    if(classifier != 'svm'):
        predictions_test = model.predict_proba(test_X)
        predictions_train = model.predict_proba(train_X)

    #only for svc
    else:
        predictions_test = model._predict_proba_lr(test_X)
        predictions_train = model._predict_proba_lr(train_X)
        
    for th in numpy.arange(0.1,1.1,0.1):
        predictions_class_Train = (predictions_train[:,1] >= th).astype('int')
        predictions_class_Test = (predictions_test[:,1] >= th).astype('int')

        report_performance(file,time,dataType,predictions_class_Train,predictions_class_Test, type_class,th, predictions_train[:,1], predictions_test[:,1],train_y,train_X.shape[0])

    # add new data
    
    return 0, 1

In [21]:
dtypes = {
    'comment_text':'str',
    'comment_id':'str',
    'page_id': 'str',
    'parent_id': 'str', 
    'post_id': 'str',
    'comment_reaction_count':'float',
    'comment_reply_count':'float',
    'comment_reaction1':'str',
    'comment_reaction2':'str',
    'comment_reaction3':'str',
    'comment_topfan':'float',
    'like':'float',
    'super_like':'float',
    'sup_like':'float',
    'angry':'float',
    'super_angry':'float',
    'sup_angry':'float',
    'love':'float',
    'super_love':'float',
    'sup_love':'float',
    'haha':'float',
    'super_haha':'float',
    'sup_haha':'float',
    'sad':'float',
    'super_sad':'float',
    'sup_sad':'float',
    'wow':'float',
    'super_wow':'float',
    'sup_wow':'float',
    'is_skoi':'float',
    'thread_level': 'float',
    'post_image': 'float',
    'post_video': 'float',
    'post_comment_count':'float',
    'post_share_count':'float',
    'post_reaction_count':'float',
    'post_like': 'float',
    'post_love': 'float',
    'post_haha': 'float', 
    'post_sad': 'float',
    'post_wow': 'float',
    'post_angry': 'float',
    'comment_image':'float',
    'comment_video':'float',
    'comment_year':'float',
    'comment_month': 'float',
    'comment_day': 'float',
    'comment_hour': 'float',
    'comment_minute': 'float',
    'comment_second': 'float',
    'comment_day_of_week': 'float',
    'post_year':'float',
    'post_month':'float',
    'post_day':'float',
    'post_hour': 'float',
    'post_minute': 'float',
    'post_second': 'float',
    'post_day_of_week': 'float',
    'comment_text_char_len': 'float',
    'comment_num_hashtag': 'float',
    'comment_avg_char_hashtag':'float',
    'comment_urls_len':'float',
    'comment_neg_emo_count':'float',
    'comment_neu_emo_count': 'float',
    'comment_pos_emo_count': 'float',
    'comment_author': 'str',
    'cue_word_rude_freq': 'float',
    'cue_word_rude_count': 'float',
    'cue_word_rude_rel_loc_first': 'float',
    'cue_word_rude_rel_loc_avg': 'float',
    'cue_word_figurative_freq': 'float',
    'cue_word_figurative_count': 'float',
    'cue_word_figurative_rel_loc_first': 'float',
    'cue_word_figurative_rel_loc_avg': 'float',
    'cue_word_offensive_freq': 'float',
    'cue_word_offensive_count': 'float',
    'cue_word_offensive_rel_loc_first': 'float',
    'cue_word_offensive_rel_loc_avg': 'float',
    'cue_word_dirty_freq': 'float',
    'cue_word_dirty_count': 'float',
    'cue_word_dirty_rel_loc_first': 'float',
    'cue_word_dirty_rel_loc_avg': 'float',
    'cue_word_conj_freq': 'float',
    'cue_word_conj_count': 'float',
    'cue_word_conj_rel_loc_first': 'float',
    'cue_word_conj_rel_loc_avg': 'float'
}
#,
context_features = [
    'comment_reaction_count','comment_reply_count', 'comment_reaction1','comment_reaction2','comment_reaction3',
    'comment_topfan','like','super_like','sup_like','angry','super_angry','sup_angry','love','super_love','sup_love',
    'haha','super_haha','sup_haha','sad','super_sad','sup_sad','wow','super_wow','sup_wow','is_skoi',
    'thread_level','post_image','post_video','post_comment_count','post_share_count','post_reaction_count',
    'post_like','post_love','post_haha','post_sad','post_wow','post_angry',
    'comment_image','comment_video','comment_year','comment_month','comment_day','comment_hour',
    'comment_minute','comment_second','comment_day_of_week','post_year','post_month',
    'post_day','post_hour','post_minute','post_second','post_day_of_week','comment_text_char_len',
    'comment_num_hashtag','comment_avg_char_hashtag','comment_urls_len','comment_neg_emo_count',
    'comment_neu_emo_count','comment_pos_emo_count', 'cue_word_rude_freq',
    'cue_word_rude_count',
    'cue_word_rude_rel_loc_first',
    'cue_word_rude_rel_loc_avg',
    'cue_word_figurative_freq',
    'cue_word_figurative_count',
    'cue_word_figurative_rel_loc_first',
    'cue_word_figurative_rel_loc_avg',
    'cue_word_offensive_freq' ,
    'cue_word_offensive_count',
    'cue_word_offensive_rel_loc_first',
    'cue_word_offensive_rel_loc_avg',
    'cue_word_dirty_freq',
    'cue_word_dirty_count',
    'cue_word_dirty_rel_loc_first',
    'cue_word_dirty_rel_loc_avg',
    'cue_word_conj_freq',
    'cue_word_conj_count',
    'cue_word_conj_rel_loc_first',
    'cue_word_conj_rel_loc_avg'
]
#comment_author
content_features = ['comment_text']
labels = ['rude','figurative','offensive','dirty']

reaction_map = {
    'haha': 1,
    'like': 2,
    'angry': 3,
    'sad': 4,
    'wow': 5,
    'love': 6,
    '<NONE>': 7
}

In [22]:
content_surround_features = ['parent_id']

In [23]:
Corpus = pd.read_csv('../../MapLabeledData/UpdateLabel/dirty_vs_non_v2.csv', dtype=dtypes)

In [24]:
reaction_columns = [f'comment_reaction{i}' for i in range(1, 4)]
Corpus[reaction_columns] = Corpus[reaction_columns].replace(reaction_map)

In [25]:
Train_X, Test_X, Train_Y, Test_Y = train_test_split(
    Corpus[content_features + context_features + content_surround_features], 
    Corpus[labels], test_size=0.2, random_state=1000);


In [26]:
Tfidf_vect = TfidfVectorizer(tokenizer=split_word, min_df=2, max_features=10000, token_pattern=None)
Tfidf_vect.fit(Train_X['comment_text'])
    
Train_X_Tfidf = Tfidf_vect.transform(Train_X['comment_text'])
Test_X_Tfidf = Tfidf_vect.transform(Test_X['comment_text'])

In [27]:
content_parent_train=[];
content_parent_train_len=[];
for i,v in enumerate(Train_X['parent_id']):
    w = Corpus[Corpus['post_id'] == v]
    w = w.append(Corpus[Corpus['comment_id'] == v])
    if(w.empty):
        content_parent_train.append('')
        content_parent_train_len.append(1)
    else:
#         w = w.iloc[0]
        l = '';
        count = 0;
        for i,v in enumerate(w['comment_text']):
            if(count < 5):
                l = l+v
                count+=1;
        content_parent_train.append(l)
        content_parent_train_len.append(count)

Train_X['comments_of_parent'] = content_parent_train

In [28]:
content_parent_test=[];
content_parent_test_len=[];
for i,v in enumerate(Test_X['parent_id']):
    w = Corpus[Corpus['post_id'] == v]
    w = w.append(Corpus[Corpus['comment_id'] == v])
    if(w.empty):
        content_parent_test.append('')
        content_parent_test_len.append(1)
    else:
#         w = w.iloc[0]
#         content_parent_test.append(w['comments'])
        l = '';
        count = 0;
        for i,v in enumerate(w['comment_text']):
            if(count < 5):
                l = l+v
                count+=1;
        content_parent_test.append(l)
        content_parent_test_len.append(count)

Test_X['comments_of_parent'] = content_parent_test

In [29]:
Tfidf_vect = TfidfVectorizer(tokenizer=split_word, min_df=2, max_features=10000, token_pattern=None)
Tfidf_vect.fit(Train_X['comments_of_parent'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=10000,
                min_df=2, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern=None,
                tokenizer=<function split_word at 0x7fe0b18b6cb0>, use_idf=True,
                vocabulary=None)

In [30]:
Train_X_Tfidf_parent = Tfidf_vect.transform(Train_X['comments_of_parent'])
Test_X_Tfidf_parent = Tfidf_vect.transform(Test_X['comments_of_parent'])

In [31]:
for i in range(0,len(content_parent_train_len)):
    Train_X_Tfidf_parent[i,:] = Train_X_Tfidf_parent[i,:]/content_parent_train_len[i]

for i in range(0,len(content_parent_test_len)):
    Test_X_Tfidf_parent[i,:] = Test_X_Tfidf_parent[i,:]/content_parent_test_len[i]

In [33]:
from scipy.sparse import hstack, csr_matrix, vstack
train_feat_context = hstack([Train_X[context_features].values, Train_X_Tfidf_parent]).toarray()
test_feat_context = hstack([Test_X[context_features].values, Test_X_Tfidf_parent]).toarray()

train_feat_content = Train_X_Tfidf
test_feat_content = Test_X_Tfidf

In [31]:
from scipy.sparse import hstack, csr_matrix, vstack
file = '80_20_dirty_tfidf_knn.csv'
createGeneralEvaluation(file)
label = 'dirty'
    
_train_y = Train_Y[label].values
_test_y = Test_Y[label].values
content_train_y = _train_y

_cp, _cn = train_and_label(
train_feat_content, content_train_y, 
test_feat_content, _test_y,
file, label, 0, "content",'knn')

Test data of content
dirty Accuracy score: 96.64281783159053
Confusion Matrix: 
 [[3485   24]
 [  98   27]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.97      0.99      0.98      3509
         1.0       0.53      0.22      0.31       125

    accuracy                           0.97      3634
   macro avg       0.75      0.60      0.64      3634
weighted avg       0.96      0.97      0.96      3634

Area under the curve ROC:  0.6048298660587061
Area under the curve PR:  0.3973768125752327
-----------------------------------------------------------

Train data of content
dirty Accuracy score: 99.52528379772961
Confusion Matrix: 
 [[13996    69]
 [    0   470]]
PRF: 
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     14065
         1.0       0.87      1.00      0.93       470

    accuracy                           1.00     14535
   macro avg       0.94      1.00      0.96     14535
weighted avg  

PRF: 
               precision    recall  f1-score   support

         0.0       0.97      1.00      0.98     14065
         1.0       1.00      0.06      0.11       470

    accuracy                           0.97     14535
   macro avg       0.98      0.53      0.55     14535
weighted avg       0.97      0.97      0.96     14535

Area under the curve ROC:  0.9977578265046024
Area under the curve PR:  0.918111554099396
___________________________________________________________________________________________________________

Test data of content
dirty Accuracy score: 96.72537149146946
Confusion Matrix: 
 [[3509    0]
 [ 119    6]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.97      1.00      0.98      3509
         1.0       1.00      0.05      0.09       125

    accuracy                           0.97      3634
   macro avg       0.98      0.52      0.54      3634
weighted avg       0.97      0.97      0.95      3634

Area under the curve ROC:

In [34]:
from scipy.sparse import hstack, csr_matrix, vstack
file = '80_20_context_dirty_knn.csv'
createGeneralEvaluation(file)
label = 'dirty'
    
_train_y = Train_Y[label].values
_test_y = Test_Y[label].values
context_train_y = _train_y

_cp, _cn = train_and_label(
train_feat_context, context_train_y, 
test_feat_context, _test_y,
file, label, 0, "context","knn")

Test data of context
dirty Accuracy score: 93.36818932306
Confusion Matrix: 
 [[3310  199]
 [  42   83]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.99      0.94      0.96      3509
         1.0       0.29      0.66      0.41       125

    accuracy                           0.93      3634
   macro avg       0.64      0.80      0.69      3634
weighted avg       0.96      0.93      0.95      3634

Area under the curve ROC:  0.810171558848675
Area under the curve PR:  0.45820989811518376
-----------------------------------------------------------

Train data of context
dirty Accuracy score: 96.53938768489853
Confusion Matrix: 
 [[13562   503]
 [    0   470]]
PRF: 
               precision    recall  f1-score   support

         0.0       1.00      0.96      0.98     14065
         1.0       0.48      1.00      0.65       470

    accuracy                           0.97     14535
   macro avg       0.74      0.98      0.82     14535
weighted avg     

PRF: 
               precision    recall  f1-score   support

         0.0       0.97      1.00      0.99      3509
         1.0       0.80      0.22      0.35       125

    accuracy                           0.97      3634
   macro avg       0.89      0.61      0.67      3634
weighted avg       0.97      0.97      0.96      3634

Area under the curve ROC:  0.810171558848675
Area under the curve PR:  0.45820989811518376
-----------------------------------------------------------

Train data of context
dirty Accuracy score: 97.64017887856897
Confusion Matrix: 
 [[14065     0]
 [  343   127]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.98      1.00      0.99     14065
         1.0       1.00      0.27      0.43       470

    accuracy                           0.98     14535
   macro avg       0.99      0.64      0.71     14535
weighted avg       0.98      0.98      0.97     14535

Area under the curve ROC:  0.990083729795554
Area under the curve P

In [35]:
from scipy.sparse import hstack, csr_matrix, vstack
file = '80_20_context_dirty_randomforest.csv'
createGeneralEvaluation(file)
label = 'dirty'
    
_train_y = Train_Y[label].values
_test_y = Test_Y[label].values
context_train_y = _train_y

_cp, _cn = train_and_label(
train_feat_context, context_train_y, 
test_feat_context, _test_y,
file, label, 0, "context","randomforest")

Test data of context
dirty Accuracy score: 92.95542102366538
Confusion Matrix: 
 [[3264  245]
 [  11  114]]
PRF: 
               precision    recall  f1-score   support

         0.0       1.00      0.93      0.96      3509
         1.0       0.32      0.91      0.47       125

    accuracy                           0.93      3634
   macro avg       0.66      0.92      0.72      3634
weighted avg       0.97      0.93      0.95      3634

Area under the curve ROC:  0.9674220575662581
Area under the curve PR:  0.7376553486032749
-----------------------------------------------------------

Train data of context
dirty Accuracy score: 97.56449948400413
Confusion Matrix: 
 [[13711   354]
 [    0   470]]
PRF: 
               precision    recall  f1-score   support

         0.0       1.00      0.97      0.99     14065
         1.0       0.57      1.00      0.73       470

    accuracy                           0.98     14535
   macro avg       0.79      0.99      0.86     14535
weighted avg  

Train data of context
dirty Accuracy score: 97.91537667698658
Confusion Matrix: 
 [[14065     0]
 [  303   167]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.98      1.00      0.99     14065
         1.0       1.00      0.36      0.52       470

    accuracy                           0.98     14535
   macro avg       0.99      0.68      0.76     14535
weighted avg       0.98      0.98      0.97     14535

Area under the curve ROC:  0.9999966719864459
Area under the curve PR:  0.9999023524036299
___________________________________________________________________________________________________________

Test data of context
dirty Accuracy score: 96.67033571821683
Confusion Matrix: 
 [[3509    0]
 [ 121    4]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.97      1.00      0.98      3509
         1.0       1.00      0.03      0.06       125

    accuracy                           0.97      3634
   macro avg       0

In [36]:
from scipy.sparse import hstack, csr_matrix, vstack
file = '80_20_context_dirty_logisticRegression.csv'
createGeneralEvaluation(file)
label = 'dirty'
    
_train_y = Train_Y[label].values
_test_y = Test_Y[label].values
context_train_y = _train_y

_cp, _cn = train_and_label(
train_feat_context, context_train_y, 
test_feat_context, _test_y,
file, label, 0, "context","logisticRegression")

/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Test data of context
dirty Accuracy score: 88.66263070996148
Confusion Matrix: 
 [[3124  385]
 [  27   98]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.99      0.89      0.94      3509
         1.0       0.20      0.78      0.32       125

    accuracy                           0.89      3634
   macro avg       0.60      0.84      0.63      3634
weighted avg       0.96      0.89      0.92      3634

Area under the curve ROC:  0.9185431746936449
Area under the curve PR:  0.23100846216687912
-----------------------------------------------------------

Train data of context
dirty Accuracy score: 88.88888888888889
Confusion Matrix: 
 [[12545  1520]
 [   95   375]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.99      0.89      0.94     14065
         1.0       0.20      0.80      0.32       470

    accuracy                           0.89     14535
   macro avg       0.60      0.84      0.63     14535
weighted avg 

PRF: 
               precision    recall  f1-score   support

         0.0       0.97      1.00      0.98     14065
         1.0       0.14      0.00      0.00       470

    accuracy                           0.97     14535
   macro avg       0.56      0.50      0.49     14535
weighted avg       0.94      0.97      0.95     14535

Area under the curve ROC:  0.920910665527074
Area under the curve PR:  0.2481008360046866
___________________________________________________________________________________________________________

Test data of context
dirty Accuracy score: 96.56026417171161
Confusion Matrix: 
 [[3508    1]
 [ 124    1]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.97      1.00      0.98      3509
         1.0       0.50      0.01      0.02       125

    accuracy                           0.97      3634
   macro avg       0.73      0.50      0.50      3634
weighted avg       0.95      0.97      0.95      3634

Area under the curve ROC:

/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

In [37]:
from scipy.sparse import hstack, csr_matrix, vstack
file = '80_20_context_dirty_mlp.csv'
createGeneralEvaluation(file)
label = 'dirty'
    
_train_y = Train_Y[label].values
_test_y = Test_Y[label].values
context_train_y = _train_y

_cp, _cn = train_and_label(
train_feat_context, context_train_y, 
test_feat_context, _test_y,
file, label, 0, "context","mlp")

Test data of context
dirty Accuracy score: 89.81838194826636
Confusion Matrix: 
 [[3166  343]
 [  27   98]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.99      0.90      0.94      3509
         1.0       0.22      0.78      0.35       125

    accuracy                           0.90      3634
   macro avg       0.61      0.84      0.65      3634
weighted avg       0.97      0.90      0.92      3634

Area under the curve ROC:  0.9158050726702764
Area under the curve PR:  0.5121198847031734
-----------------------------------------------------------

Train data of context
dirty Accuracy score: 90.62263501891985
Confusion Matrix: 
 [[12794  1271]
 [   92   378]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.99      0.91      0.95     14065
         1.0       0.23      0.80      0.36       470

    accuracy                           0.91     14535
   macro avg       0.61      0.86      0.65     14535
weighted avg  

PRF: 
               precision    recall  f1-score   support

         0.0       0.99      0.94      0.96      3509
         1.0       0.29      0.70      0.41       125

    accuracy                           0.93      3634
   macro avg       0.64      0.82      0.69      3634
weighted avg       0.96      0.93      0.94      3634

Area under the curve ROC:  0.9158050726702764
Area under the curve PR:  0.5121198847031734
-----------------------------------------------------------

Train data of context
dirty Accuracy score: 93.6842105263158
Confusion Matrix: 
 [[13275   790]
 [  128   342]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.99      0.94      0.97     14065
         1.0       0.30      0.73      0.43       470

    accuracy                           0.94     14535
   macro avg       0.65      0.84      0.70     14535
weighted avg       0.97      0.94      0.95     14535

Area under the curve ROC:  0.9206599299604419
Area under the curve P

In [38]:
from scipy.sparse import hstack, csr_matrix, vstack
file = '80_20_context_dirty_tree.csv'
createGeneralEvaluation(file)
label = 'dirty'
    
_train_y = Train_Y[label].values
_test_y = Test_Y[label].values
context_train_y = _train_y

_cp, _cn = train_and_label(
train_feat_context, context_train_y, 
test_feat_context, _test_y,
file, label, 0, "context","tree")

Test data of context
dirty Accuracy score: 97.33076499724821
Confusion Matrix: 
 [[3463   46]
 [  51   74]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.99      0.99      0.99      3509
         1.0       0.62      0.59      0.60       125

    accuracy                           0.97      3634
   macro avg       0.80      0.79      0.80      3634
weighted avg       0.97      0.97      0.97      3634

Area under the curve ROC:  0.7894454260473069
Area under the curve PR:  0.6113503944230416
-----------------------------------------------------------

Train data of context
dirty Accuracy score: 100.0
Confusion Matrix: 
 [[14065     0]
 [    0   470]]
PRF: 
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     14065
         1.0       1.00      1.00      1.00       470

    accuracy                           1.00     14535
   macro avg       1.00      1.00      1.00     14535
weighted avg       1.00   

PRF: 
               precision    recall  f1-score   support

         0.0       0.99      0.99      0.99      3509
         1.0       0.62      0.59      0.60       125

    accuracy                           0.97      3634
   macro avg       0.80      0.79      0.80      3634
weighted avg       0.97      0.97      0.97      3634

Area under the curve ROC:  0.7894454260473069
Area under the curve PR:  0.6113503944230416
-----------------------------------------------------------

Train data of context
dirty Accuracy score: 100.0
Confusion Matrix: 
 [[14065     0]
 [    0   470]]
PRF: 
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     14065
         1.0       1.00      1.00      1.00       470

    accuracy                           1.00     14535
   macro avg       1.00      1.00      1.00     14535
weighted avg       1.00      1.00      1.00     14535

Area under the curve ROC:  1.0
Area under the curve PR:  1.0
__________________

In [39]:
from scipy.sparse import hstack, csr_matrix, vstack
file = '80_20_context_dirty_svm.csv'
createGeneralEvaluation(file)
label = 'dirty'
    
_train_y = Train_Y[label].values
_test_y = Test_Y[label].values
context_train_y = _train_y

_cp, _cn = train_and_label(
train_feat_context, context_train_y, 
test_feat_context, _test_y,
file, label, 0, "context","svm")

/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Test data of context
dirty Accuracy score: 56.52173913043478
Confusion Matrix: 
 [[1931 1578]
 [   2  123]]
PRF: 
               precision    recall  f1-score   support

         0.0       1.00      0.55      0.71      3509
         1.0       0.07      0.98      0.13       125

    accuracy                           0.57      3634
   macro avg       0.54      0.77      0.42      3634
weighted avg       0.97      0.57      0.69      3634

Area under the curve ROC:  0.9326645768025079
Area under the curve PR:  0.2668507526494854
-----------------------------------------------------------

Train data of context
dirty Accuracy score: 56.917784657722734
Confusion Matrix: 
 [[7814 6251]
 [  11  459]]
PRF: 
               precision    recall  f1-score   support

         0.0       1.00      0.56      0.71     14065
         1.0       0.07      0.98      0.13       470

    accuracy                           0.57     14535
   macro avg       0.53      0.77      0.42     14535
weighted avg     

/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

In [11]:
from scipy.sparse import hstack, csr_matrix, vstack
file = 'combined_tfidf_dirty_tree.csv'
createGeneralEvaluation(file)
label = 'dirty'

train_feat_content = Train_X_Tfidf
test_feat_content = Test_X_Tfidf
    
train_feat_context = Train_X[context_features].values
test_feat_context = Test_X[context_features].values

train_feat = hstack([train_feat_content,train_feat_context])
test_feat = hstack([test_feat_content,test_feat_context])
    
_train_y = Train_Y[label].values
_test_y = Test_Y[label].values
train_y = _train_y

_cp, _cn = train_and_label(
train_feat, train_y, 
test_feat, _test_y,
file, label, 0, "combined",'tree')

(14535, 8627)
(14535, 8687)
Test data of combined
dirty Accuracy score: 98.0462300495322
Confusion Matrix: 
 [[3475   34]
 [  37   88]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.99      0.99      0.99      3509
         1.0       0.72      0.70      0.71       125

    accuracy                           0.98      3634
   macro avg       0.86      0.85      0.85      3634
weighted avg       0.98      0.98      0.98      3634

Area under the curve ROC:  0.8471553149045311
Area under the curve PR:  0.7177465467307848
-----------------------------------------------------------

Train data of combined
dirty Accuracy score: 100.0
Confusion Matrix: 
 [[14065     0]
 [    0   470]]
PRF: 
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     14065
         1.0       1.00      1.00      1.00       470

    accuracy                           1.00     14535
   macro avg       1.00      1.00      1.00     145

PRF: 
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     14065
         1.0       1.00      1.00      1.00       470

    accuracy                           1.00     14535
   macro avg       1.00      1.00      1.00     14535
weighted avg       1.00      1.00      1.00     14535

Area under the curve ROC:  1.0
Area under the curve PR:  1.0
___________________________________________________________________________________________________________

Test data of combined
dirty Accuracy score: 98.0462300495322
Confusion Matrix: 
 [[3475   34]
 [  37   88]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.99      0.99      0.99      3509
         1.0       0.72      0.70      0.71       125

    accuracy                           0.98      3634
   macro avg       0.86      0.85      0.85      3634
weighted avg       0.98      0.98      0.98      3634

Area under the curve ROC:  0.8471553149045311
Area und